![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/configuration.png)

Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Configuration

_**Setting up your Azure Machine Learning services workspace and configuring your notebook library**_

---
---

## Table of Contents

1. [Introduction](#Introduction)
    1. What is an Azure Machine Learning workspace
1. [Setup](#Setup)
    1. Azure subscription
    1. Azure ML SDK and other library installation
    1. Azure Container Instance registration
1. [Configure your Azure ML Workspace](#Configure-your-Azure-ML-workspace)
    1. Workspace parameters
    1. Access your workspace
    1. Create a new workspace
1. [Next steps](#Next_steps)
1. [The end](#The-End)

---

## Introduction

This notebook configures your library of notebooks to connect to an Azure Machine Learning (ML) workspace.  In this case, a library contains all of the notebooks in the current folder and any nested folders.  You can configure this notebook library to use an existing workspace or create a new workspace. _聽起來就是用整個 folder (directory) 來與 AutoML workspace 連結。_ 

Typically you will need to run this notebook only once per notebook library as all other notebooks will use connection information that is written here.  If you want to redirect your notebook library to work with a different workspace, then you should re-run this notebook. _與這個 folder 連結的 workspace 還可以抽換_

In this notebook you will
* Learn about getting an Azure subscription  _看看無妨，咱好像用不到？ subscription 1 是公司給的好像_
* Specify your workspace parameters 
* Access or create your workspace _[這個咱之前做過成功了](OneDrive/%E6%96%87%E4%BB%B6/Jupyter%20Notebooks/Azure/What%20is%20the%20Azure%20Machine%20Learning%20SDK%20for%20Python.ipynb)_
* Add a default compute cluster for your workspace _哈！上面那篇 tutor 沒教好的這下有救了？_

### What is an Azure Machine Learning workspace

An Azure ML Workspace is an Azure resource that organizes and coordinates the actions of many other Azure resources to assist in executing and sharing machine learning workflows.  In particular, an Azure ML Workspace coordinates storage, databases, and compute resources providing added functionality for machine learning experimentation, deployment, inference, and the monitoring of deployed models.

## Setup

This section describes activities required before you can access any Azure ML services functionality.

### 1. Azure Subscription

In order to create an Azure ML Workspace, first you need access to an Azure subscription.  An Azure subscription allows you to manage storage, compute, and other assets in the Azure cloud.  You can [create a new subscription](https://azure.microsoft.com/en-us/free/) or access existing subscription information from the [Azure portal](https://portal.azure.com).  Later in this notebook you will need information such as your subscription ID in order to create and access AML workspaces.

### 2. Azure ML SDK and other library installation

If you are running in your own environment, follow [SDK installation instructions](https://docs.microsoft.com/azure/machine-learning/service/how-to-configure-environment).  If you are running in Azure Notebooks or another Microsoft managed environment, the SDK is already installed.

Also install following libraries to your environment. Many of the example notebooks depend on them

```
(myenv) $ conda install -y matplotlib tqdm scikit-learn
```

Once installation is complete, the following cell checks the Azure ML SDK version:

In [ ]:
import azureml.core

print("This notebook was created using version 1.39.0 of the Azure ML SDK")
print("You are currently using version", azureml.core.VERSION, "of the Azure ML SDK")

If you are using an older version of the SDK then this notebook was created using, you should upgrade your SDK.

### 3. Azure Container Instance registration
Azure Machine Learning uses of [Azure Container Instance (ACI)](https://azure.microsoft.com/services/container-instances) to deploy dev/test web services. An Azure subscription needs to be registered to use ACI.  If you or the subscription owner have not yet registered ACI on your subscription, you will need to use the [Azure CLI](https://docs.microsoft.com/en-us/cli/azure/install-azure-cli?view=azure-cli-latest) and execute the following commands.  Note that if you ran through the AML [quickstart](https://docs.microsoft.com/en-us/azure/machine-learning/service/quickstart-get-started) you have already registered ACI. 

```shell
# check to see if ACI is already registered
(myenv) $ az provider show -n Microsoft.ContainerInstance -o table

# if ACI is not registered, run this command.
# note you need to be the subscription owner in order to execute this command successfully.
(myenv) $ az provider register -n Microsoft.ContainerInstance
```

---

## Configure your Azure ML workspace

### Workspace parameters

To use an AML Workspace, you will need to import the Azure ML SDK and supply the following information:
* Your subscription id
* A resource group name
* (optional) The region that will host your workspace
* A name for your workspace

You can get your subscription ID from the [Azure portal](https://portal.azure.com).

You will also need access to a [_resource group_](https://docs.microsoft.com/en-us/azure/azure-resource-manager/resource-group-overview#resource-groups), which organizes Azure resources and provides a default region for the resources in a group.  You can see what resource groups to which you have access, or create a new one in the [Azure portal](https://portal.azure.com).  If you don't have a resource group, the create workspace command will create one for you using the name you provide.

The region to host your workspace will be used if you are creating a new workspace.  You do not need to specify this if you are using an existing workspace. You can find the list of supported regions [here](https://azure.microsoft.com/en-us/global-infrastructure/services/?products=machine-learning-service).  You should pick a region that is close to your location or that contains your data.

The name for your workspace is unique within the subscription and should be descriptive enough to discern among other AML Workspaces.  The subscription may be used only by you, or it may be used by your department or your entire enterprise, so choose a name that makes sense for your situation.

The following cell allows you to specify your workspace parameters.  This cell uses the python method `os.getenv` to read values from environment variables which is useful for automation.  If no environment variable exists, the parameters will be set to the specified default values.  

If you ran the Azure Machine Learning [quickstart](https://docs.microsoft.com/en-us/azure/machine-learning/service/quickstart-get-started) in Azure Notebooks, you already have a configured workspace!  You can go to your Azure Machine Learning Getting Started library, view *config.json* file, and copy-paste the values for subscription ID, resource group and workspace name below.

Replace the default values in the cell below with your workspace parameters

In [ ]:
import os

subscription_id = os.getenv("SUBSCRIPTION_ID", default="<my-subscription-id>")
resource_group = os.getenv("RESOURCE_GROUP", default="<my-resource-group>")
workspace_name = os.getenv("WORKSPACE_NAME", default="<my-workspace-name>")
workspace_region = os.getenv("WORKSPACE_REGION", default="eastus2")

### Access your workspace

The following cell uses the Azure ML SDK to attempt to load the workspace specified by your parameters.  If this cell succeeds, your notebook library will be configured to access the workspace from all notebooks using the `Workspace.from_config()` method.  The cell can fail if the specified workspace doesn't exist or you don't have permissions to access it. 

In [ ]:
from azureml.core import Workspace

try:
    ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
    # write the details of the workspace to a configuration file to the notebook library
    ws.write_config()
    print("Workspace configuration succeeded. Skip the workspace creation steps below")
except:
    print("Workspace not accessible. Change your parameters or create a new workspace below")

### Create a new workspace

If you don't have an existing workspace and are the owner of the subscription or resource group, you can create a new workspace.  If you don't have a resource group, the create workspace command will create one for you using the name you provide.

**Note**: As with other Azure services, there are limits on certain resources (for example AmlCompute quota) associated with the Azure ML service. Please read [this article](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-manage-quotas) on the default limits and how to request more quota.

This cell will create an Azure ML workspace for you in a subscription provided you have the correct permissions.

This will fail if:
* You do not have permission to create a workspace in the resource group
* You do not have permission to create a resource group if it's non-existing.
* You are not a subscription owner or contributor and no Azure ML workspaces have ever been created in this subscription

If workspace creation fails, please work with your IT admin to provide you with the appropriate permissions or to provision the required resources.


In [ ]:
from azureml.core import Workspace

# Create the workspace using the specified parameters
ws = Workspace.create(name = workspace_name,
                      subscription_id = subscription_id,
                      resource_group = resource_group, 
                      location = workspace_region,
                      create_resource_group = True,
                      exist_ok = True)
ws.get_details()

# write the details of the workspace to a configuration file to the notebook library
ws.write_config()

---

## Next_steps

In this notebook you configured this notebook library to connect easily to an Azure ML workspace.  You can copy this notebook to your own libraries to connect them to you workspace, or use it to bootstrap new workspaces completely.

If you came here from another notebook, you can return there and complete that exercise, or you can try out the [Tutorials](./tutorials) or jump into "how-to" notebooks and start creating and deploying models.  A good place to start is the [train within notebook](./how-to-use-azureml/training/train-within-notebook) example that walks through a simplified but complete end to end machine learning process.

# The End